<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Pardis Zahraei
### Student Number : 99109777
### Colab Link: https://colab.research.google.com/drive/1TpquJADYHp06dyrwvw4vILUD37wG_y-3?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [4]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-05 19:31:28--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-05 19:31:29--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc317545976517b96a1e5a205b6a.dl.dropboxusercontent.com/cd/0/get/B7fEVzBDlvY09ozCj5GCjtt9pIfg55qO3aPJP_04vJq-s5I3gCLKBhD-QGCu-oR0nig5PabS89tsp1jQ2ru5BbonZYWlGsnblOSQg_hxjAokYyvGwtAAELQqGlfT5KEofTsLpNt9kNlyfI0EoixGxhBj9U1rTc6U7NXB1z45kw7m4xElT0aCUeTdz_isaYoKWvc/file?dl=1# [following]
--2023-05-05 19:31:29--  https://uc317545976517b96a1e5a205b6a.dl.dropboxusercontent.com/cd/0/get/B7fEVzBDlvY09ozCj5GCjtt9pIfg55qO3aPJP_04vJ

In [5]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

mkdir: cannot create directory ‘/content/dataset/’: File exists
Archive:  /content/dataset.zip
replace /content/dataset/content/news-train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Preprocessing

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [45]:
import pandas as pd
csv_path = '/content/dataset/content/news-train.csv'
df = pd.read_csv(csv_path)
print(df.head())

       topic                                               text
0     sports  به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1   politics  پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2   politics  # دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3  economics  کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4  economics  به گزارش ایسنا صحبت با برخی # # آن است که امرو...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [46]:
topic_counts = df['topic'].value_counts()
print(topic_counts)

sports       123105
economics     49600
cultural      13359
politics      11297
Name: topic, dtype: int64


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [47]:
min_count = df['topic'].value_counts().min()
dfs = []
for topic in df['topic'].unique():
    temp = df[df['topic'] == topic].sample(n=min_count, random_state=42)
    dfs.append(temp)
balanced_df = pd.concat(dfs)

# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [48]:
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [49]:
train_df['tokens'] = train_df['text'].str.split()
print(train_df['tokens'][0])

['به', 'گزارش', 'ورزش', 'سه', 'تیم', 'فوتبال', 'پدیده', 'در', 'روزی', 'که', 'انتظار', 'می\u200cرفت', 'با', 'توجه', 'به', 'مشکلات', 'و', 'حواشی', 'که', 'در', 'طی', 'هفته', 'گذشته', 'برای', 'نفت', 'مسجد', 'به', 'وجود', 'آمده', 'است', 'به\u200cراحتی', 'به', 'جمع', 'هشت', 'تیم', 'نهایی', 'صعود', 'کند', '#', 'یک', '#', '#', 'باز', 'شد', 'و', 'در', 'ادامه', 'نتوانستند', '#', 'خورده', 'را', 'جبران', 'کنند', 'تا', 'در', 'شهر', 'مشهد', 'شکست', '#', 'را', 'متحمل', 'شوند', 'یحیی', 'گل\u200cمحمدی', 'سرمربی', 'تیم', 'فوتبال', 'شهر', 'خودرو', 'پس', 'از', 'پایان', 'بازی', 'در', 'نشست', 'خبری', 'حضور', 'یافت', 'و', 'در', 'ابتدا', 'بیان', 'کرد', 'نتیجه', 'آن', 'چیزی', 'نبود', 'که', '#', 'را', 'داشتیم', '#', 'جام', 'حذفی', 'جام', 'کم', 'اشتباه', 'بودن', 'است', 'یک', 'اشتباه', '#', 'داشتیم', 'و', 'با', 'گلی', 'که', '#', 'جریمه', 'شدیم', 'حریف', 'با', '#', 'دفاع', 'منظم', 'و', 'فشرده', 'کار', 'را', 'سخت', 'کرد', 'در', 'این', 'مسابقات', 'باید', 'از', 'تک', '#', 'استفاده', 'کنید', 'در', 'ادامه', 'هم', 'فشار

Count the tokens and list the top 5 frequent ones (5 points)

In [50]:
from collections import Counter
tokens_list = [token for tokens in train_df['tokens'] for token in tokens]
token_c = Counter(tokens_list)
top_tokens = token_c.most_common(5)
for token, count in top_tokens:
    print(f"{token}: {count}")

#: 1621876
و: 878306
در: 743093
به: 580896
از: 435477


Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [13]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-05 19:32:18--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-05 19:32:19--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-05 19:32:19 (8.12 MB/s)

In [51]:
with open('/content/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token != '#']
train_df['real_tokens'] = train_df['tokens'].apply(remove_stopwords)
print(train_df['real_tokens'][0])

['گزارش', 'ورزش', 'سه', 'تیم', 'فوتبال', 'پدیده', 'روزی', 'انتظار', 'می\u200cرفت', 'توجه', 'مشکلات', 'حواشی', 'طی', 'هفته', 'گذشته', 'برای', 'نفت', 'مسجد', 'وجود', 'به\u200cراحتی', 'جمع', 'هشت', 'تیم', 'نهایی', 'صعود', 'کند', 'باز', 'ادامه', 'نتوانستند', 'خورده', 'جبران', 'کنند', 'شهر', 'مشهد', 'شکست', 'متحمل', 'یحیی', 'گل\u200cمحمدی', 'سرمربی', 'تیم', 'فوتبال', 'شهر', 'خودرو', 'پایان', 'بازی', 'نشست', 'خبری', 'حضور', 'یافت', 'ابتدا', 'بیان', 'کرد', 'نتیجه', 'داشتیم', 'جام', 'حذفی', 'جام', 'کم', 'اشتباه', 'اشتباه', 'داشتیم', 'گلی', 'جریمه', 'شدیم', 'حریف', 'دفاع', 'منظم', 'فشرده', 'کار', 'سخت', 'کرد', 'این', 'مسابقات', 'باید', 'تک', 'کنید', 'ادامه', 'فشار', 'زیادی', 'آوردیم', 'نشد', 'گل', 'برسیم', 'فکر', 'می\u200cکنم', 'بازیکنان', 'تلاش', 'فراوانی', 'کردند', 'نشد', 'برای', 'نفت', 'مسجدسلیمان', 'آرزوی', 'موفقیت', 'ادامه', 'می\u200cکنیم', 'گل\u200cمحمدی', 'خصوص', 'شرایط', 'تیمش', 'لیگ', 'ایران', 'همیشه', 'وقفه', 'این', 'شرایط', 'همیشه', 'فوتبال', 'روزهای', 'کاری', 'نمی\u200cشد', 'کرد', '

Make a list of the remaining unique tokens (5 points)

In [52]:
real_tokens_list = [token for tokens in train_df['real_tokens'] for token in tokens]
unique_tokens = set(real_tokens_list)
print("First 10 unique tokens:")
for token in list(unique_tokens)[:10]:
    print(token)

First 10 unique tokens:
خورشید
وقت
اطلاع‌رسانی
مطالبات
لیورپول
عامل
ریسک
نماد
مولد
انتقادی


Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
train_df['real_text'] = train_df['real_tokens'].apply(' '.join)
vectorizer = CountVectorizer()
vectorizer.fit(train_df['real_text'])
s_xtrain = vectorizer.transform(train_df['real_text'])
x_train = s_xtrain.toarray()

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [54]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['topic'])
y_train = label_encoder.transform(train_df['topic'])

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [55]:
test_df['tokens'] = test_df['text'].str.split()
test_df['real_tokens'] = test_df['tokens'].apply(remove_stopwords)
test_df['real_text'] = test_df['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(test_df['real_text'])
x_test = s_xtest.toarray()

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [56]:
y_test = label_encoder.transform(test_df['topic'])

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [57]:
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)

MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [58]:
y_pred = nb_model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [59]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [23]:
class NaiveBayes:
    def __init__(self):
        self.classes = None
        self.priors = None
        self.c_probs = None

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.priors = np.zeros(len(self.classes))
        for i, c in enumerate(self.classes):
            self.priors[i] = np.mean(y == c)
        self.c_probs = []
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            word_c = np.sum(X_c, axis=0)
            cond_probs = (word_c + 1) / (np.sum(word_c) + len(word_c))
            self.c_probs.append(cond_probs)

    def predict(self, X):
        y_pred = []
        for x in X:
            log_probs = []
            for i, c in enumerate(self.classes):
                log_prob = np.log(self.priors[i]) + np.sum(np.log(self.c_probs[i]) * x)
                log_probs.append(log_prob)
            y_pred.append(self.classes[np.argmax(log_probs)])
        return np.array(y_pred)

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [24]:
nb = NaiveBayes()
nb.fit(x_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [25]:
y_pred = nb.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef

In [27]:
#my code
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter


csv_path = '/content/dataset/content/news-train.csv'
df = pd.read_csv(csv_path)
min_count = df['topic'].value_counts().min()
dfs = []
for topic in df['topic'].unique():
    temp_df = df[df['topic'] == topic].sample(n=min_count, random_state=42)
    dfs.append(temp_df)
balanced_df = pd.concat(dfs)
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
train_df['tokens'] = train_df['text'].str.split()
from collections import Counter
tokens_list = [token for tokens in train_df['tokens'] for token in tokens]
with open('/content/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token != '#']
train_df['real_tokens'] = train_df['tokens'].apply(remove_stopwords)
real_tokens_list = [token for tokens in train_df['real_tokens'] for token in tokens]
unique_tokens = set(real_tokens_list)
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
train_df['real_text'] = train_df['real_tokens'].apply(' '.join)
vectorizer = CountVectorizer()
vectorizer.fit(train_df['real_text'])
s_xtrain = vectorizer.transform(train_df['real_text'])
x_train = s_xtrain.toarray()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['topic'])
y_train = label_encoder.transform(train_df['topic'])
test_df['tokens'] = test_df['text'].str.split()
test_df['real_tokens'] = test_df['tokens'].apply(remove_stopwords)
test_df['real_text'] = test_df['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(test_df['real_text'])
x_test = s_xtest.toarray()
y_test = label_encoder.transform(test_df['topic'])
nb = NaiveBayes()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



In [28]:
df_test = pd.read_csv('assignment4-test-data.csv')
df_test = df_test.drop(["ID"], axis=1)
df_test['tokens'] = df_test['text'].str.split()
df_test['real_tokens'] = df_test['tokens'].apply(remove_stopwords)
df_test['real_text'] = df_test['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(df_test['real_text'])
x_test = s_xtest.toarray()
y_pred = nb.predict(x_test)
predicted_topics = label_encoder.inverse_transform(y_pred)
def map_topic(topic):
    if topic == 'politics':
        return 1
    elif topic == 'economics':
        return 2
    elif topic == 'sports':
        return 3
    elif topic == 'cultural':
        return 4
mapped_topics = np.vectorize(map_topic)(predicted_topics)
id_arr=np.arange(1,501)
print(len(mapped_topics))
print(len(id_arr))
final_df=pd.DataFrame({'topic':mapped_topics,'ID':id_arr })
final_df.to_csv('cc.csv', index=False)

500
500


In [29]:
#sklearn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter


csv_path = '/content/dataset/content/news-train.csv'
df = pd.read_csv(csv_path)
min_count = df['topic'].value_counts().min()
dfs = []
for topic in df['topic'].unique():
    temp_df = df[df['topic'] == topic].sample(n=min_count, random_state=42)
    dfs.append(temp_df)
balanced_df = pd.concat(dfs)
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
train_df['tokens'] = train_df['text'].str.split()
from collections import Counter
tokens_list = [token for tokens in train_df['tokens'] for token in tokens]
with open('/content/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token != '#']
train_df['real_tokens'] = train_df['tokens'].apply(remove_stopwords)
real_tokens_list = [token for tokens in train_df['real_tokens'] for token in tokens]
unique_tokens = set(real_tokens_list)
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
train_df['real_text'] = train_df['real_tokens'].apply(' '.join)
vectorizer = CountVectorizer()
vectorizer.fit(train_df['real_text'])
s_xtrain = vectorizer.transform(train_df['real_text'])
x_train = s_xtrain.toarray()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['topic'])
y_train = label_encoder.transform(train_df['topic'])
test_df['tokens'] = test_df['text'].str.split()
test_df['real_tokens'] = test_df['tokens'].apply(remove_stopwords)
test_df['real_text'] = test_df['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(test_df['real_text'])
x_test = s_xtest.toarray()
y_test = label_encoder.transform(test_df['topic'])
nb_mult = MultinomialNB()
nb_mult.fit(x_train, y_train)
y_pred = nb_mult.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



In [30]:
df_test = pd.read_csv('assignment4-test-data.csv')
df_test = df_test.drop(["ID"], axis=1)
df_test['tokens'] = df_test['text'].str.split()
df_test['real_tokens'] = df_test['tokens'].apply(remove_stopwords)
df_test['real_text'] = df_test['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(df_test['real_text'])
x_test = s_xtest.toarray()
y_pred = nb.predict(x_test)
predicted_topics = label_encoder.inverse_transform(y_pred)
def map_topic(topic):
    if topic == 'politics':
        return 1
    elif topic == 'economics':
        return 2
    elif topic == 'sports':
        return 3
    elif topic == 'cultural':
        return 4
mapped_topics = np.vectorize(map_topic)(predicted_topics)
id_arr=np.arange(1,501)
print(len(mapped_topics))
print(len(id_arr))
final_df=pd.DataFrame({'topic':mapped_topics,'ID':id_arr })
final_df.to_csv('cc_new.csv', index=False)

500
500


In [40]:
#my code
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
csv_path = '/content/dataset/content/news-train.csv'
df = pd.read_csv(csv_path)
min_count = df['topic'].value_counts().min()
dfs = []
for topic in df['topic'].unique():
    temp_df = df[df['topic'] == topic].sample(n=min_count, random_state=42)
    dfs.append(temp_df)
balanced_df = pd.concat(dfs)
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
train_df['tokens'] = train_df['text'].str.split()
from collections import Counter
tokens_list = [token for tokens in train_df['tokens'] for token in tokens]
with open('/content/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token != '#']
train_df['real_tokens'] = train_df['tokens'].apply(remove_stopwords)
real_tokens_list = [token for tokens in train_df['real_tokens'] for token in tokens]
unique_tokens = set(real_tokens_list)
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
train_df['real_text'] = train_df['real_tokens'].apply(' '.join)
vectorizer = CountVectorizer()
vectorizer.fit(train_df['real_text'])
s_xtrain = vectorizer.transform(train_df['real_text'])
x_train = s_xtrain.toarray()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['topic'])
y_train = label_encoder.transform(train_df['topic'])
test_df['tokens'] = test_df['text'].str.split()
test_df['real_tokens'] = test_df['tokens'].apply(remove_stopwords)
test_df['real_text'] = test_df['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(test_df['real_text'])
x_test = s_xtest.toarray()
y_test = label_encoder.transform(test_df['topic'])
nb = NaiveBayes()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



In [41]:
df_test = pd.read_csv('assignment4-test-data.csv')
df_test = df_test.drop(["ID"], axis=1)
df_test['tokens'] = df_test['text'].str.split()
df_test['real_tokens'] = df_test['tokens'].apply(remove_stopwords)
df_test['real_text'] = df_test['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(df_test['real_text'])
x_test = s_xtest.toarray()
y_pred = nb.predict(x_test)
predicted_topics = label_encoder.inverse_transform(y_pred)
def map_topic(topic):
    if topic == 'politics':
        return 1
    elif topic == 'economics':
        return 2
    elif topic == 'sports':
        return 3
    elif topic == 'cultural':
        return 4
mapped_topics = np.vectorize(map_topic)(predicted_topics)
id_arr=np.arange(1,501)
print(len(mapped_topics))
print(len(id_arr))
final_df=pd.DataFrame({'topic':mapped_topics,'ID':id_arr })
final_df.to_csv('ff.csv', index=False)

500
500


In [43]:
csv_path = '/content/dataset/content/news-train.csv'
df = pd.read_csv(csv_path)
min_count = df['topic'].value_counts().min()
dfs = []
for topic in df['topic'].unique():
    temp_df = df[df['topic'] == topic].sample(n=min_count, random_state=42)
    dfs.append(temp_df)
balanced_df = pd.concat(dfs)
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)
train_df['tokens'] = train_df['text'].str.split()
from collections import Counter
tokens_list = [token for tokens in train_df['tokens'] for token in tokens]
with open('/content/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stopwords and token != '#']
train_df['real_tokens'] = train_df['tokens'].apply(remove_stopwords)
real_tokens_list = [token for tokens in train_df['real_tokens'] for token in tokens]
unique_tokens = set(real_tokens_list)
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
train_df['real_text'] = train_df['real_tokens'].apply(' '.join)
vectorizer = CountVectorizer()
vectorizer.fit(train_df['real_text'])
s_xtrain = vectorizer.transform(train_df['real_text'])
x_train = s_xtrain.toarray()
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(train_df['topic'])
y_train = label_encoder.transform(train_df['topic'])
test_df['tokens'] = test_df['text'].str.split()
test_df['real_tokens'] = test_df['tokens'].apply(remove_stopwords)
test_df['real_text'] = test_df['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(test_df['real_text'])
x_test = s_xtest.toarray()
y_test = label_encoder.transform(test_df['topic'])
param_grid = {'alpha': np.arange(0.1, 1.1, 0.1)}
nb = MultinomialNB()
grid_search = GridSearchCV(estimator=nb, param_grid=param_grid, cv=5)
grid_search.fit(x_train, y_train)
best_alpha = grid_search.best_params_['alpha']
nb = MultinomialNB(alpha=best_alpha)
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
print(classification_report(y_test, y_pred))
df_test = pd.read_csv('assignment4-test-data.csv')
df_test = df_test.drop(["ID"], axis=1)
df_test['tokens'] = df_test['text'].str.split()
df_test['real_tokens'] = df_test['tokens'].apply(remove_stopwords)
df_test['real_text'] = df_test['real_tokens'].apply(' '.join)
s_xtest = vectorizer.transform(df_test['real_text'])
x_test = s_xtest.toarray()
y_pred = nb.predict(x_test)
predicted_topics = label_encoder.inverse_transform(y_pred)
def map_topic(topic):
    if topic == 'politics':
        return 1
    elif topic == 'economics':
        return 2
    elif topic == 'sports':
        return 3
    elif topic == 'cultural':
        return 4
mapped_topics = np.vectorize(map_topic)(predicted_topics)
id_arr=np.arange(1,501)
print(len(mapped_topics))
print(len(id_arr))
final_df=pd.DataFrame({'topic':mapped_topics,'ID':id_arr })
final_df.to_csv('eee.csv', index=False)

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038

500
500


In [44]:
print(best_alpha)

0.1
